- Skip-Gram with Negative Sampling
    - Negative Sampling은 w2v이 학습 과정에서 전체 단어 집합이 아니라 일부 단어 집합에만 집중할 수 있도록 한다.
    - 만약 현재 집중하고 있는 주변 단어가 '고양이', '귀여운'이라고 가정하고 여기에 '돈가스'와 '컴퓨터'와 같은 주변 단어가 아닌 단어들을 일부 들고와서 주변 단어들은 긍정하고 랜덤으로 샘플링 된 단어들은 부정으로 레이블링하면, 전체 단어 집합보다 훨씬 작은 단어 집합을 만들어 이진 분류하기 위한 데이터셋을 만들 수 있다. 
- SGNS 관점에서 w2v의 한계
    - w2v의 output layer에서 소프트맥스 함수를 지난 단어 집합 크기의 벡터와 실제값과의 오차를 구하고 임베딩 테이블에 있는 모든 단어에 대한 임베딩 벡터 값을 업데이트한다.
    - 단어 집합의 크기가 커질수록 학습하기에 무거운 모델이 된다.
    - 만약 현재 집중하고 있는 중심 단어와 주변 단어가 '강아지'와 '고양이' 그리고 '귀여운'과 같은 단어라면, 이 단어들과 연관 관계가 없는 '돈가스'나 '컴퓨터'와 같은 수많은 단어의 임베딩 벡터까지 업데이트하는 것은 비효율적이라는 것이다.
- SGNS가 왜 필요한가?
    - w2v보다 연산량에서 효율적이다.
        

20뉴스그룹 데이터 전처리하기

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups
from tensorflow.keras.preprocessing.text import Tokenizer

2023-03-05 16:32:16.315758: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print('총 샘플 수: ', len(documents))

총 샘플 수:  11314


In [4]:
#정제
news_df = pd.DataFrame({'document':documents})
#특수 문자 제거 (^ -> 부정표현)
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
#길이가 3이하인 단어는 제거
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
#전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

/Users/minjooshin/bin/miniconda3/envs/kubicPJ/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [5]:
#df에 null값이 존재하는지 확인
news_df.isnull().values.any()
#빈 값을 null값으로 변환하고 다시 null값 존재 여부 확인
news_df.replace("", float("NaN"), inplace=True)
news_df.isnull().values.any()

True

In [6]:
news_df.dropna(inplace=True)
print("총 샘플 수: ", len(news_df))

총 샘플 수:  10995


In [7]:
#불용어 제거
stop_words = stopwords.words('english')
# apply -> 하나의 열의 모든 행에 대한 연산
# 연산 후 tpye은 <class 'pandas.core.series.Series'>
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split())
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
#series -> list로 변환
tokenized_doc = tokenized_doc.to_list()

In [8]:
#단어 1개 이하인 샘플의 인덱스 찾아서 제거
drop_train = [index for index, sentence in enumerate(tokenized_doc) if len(sentence) <= 1]
#numpy.delete(arr, obj, axis=None), obj -> 삭제할 인덱스가 저장되어있는 sub-array
tokenized_doc = np.delete(tokenized_doc, drop_train, axis=0)
print('총 샘플 수: ', len(tokenized_doc))

총 샘플 수:  10940


/Users/minjooshin/bin/miniconda3/envs/kubicPJ/lib/python3.7/site-packages/numpy/lib/function_base.py:4454: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asarray(arr)


In [17]:
#토큰화
tokenizer = Tokenizer()
#fit_on_texts - 문자 데이터를 입력받아서 리스트의 형태로 변환
tokenizer.fit_on_texts(tokenized_doc)
#word_index - 단어(key)와 숫자(value)의 쌍을 포함하는 딕셔너리 반환
word2idx = tokenizer.word_index
#숫자가 key이고 단어가 value인 형태로 저장 
idx2word = {value : key for key, value in word2idx.items()}

#정수 인코딩
encoded = tokenizer.texts_to_sequences(tokenized_doc)

[list(['well', 'sure', 'story', 'seem', 'biased', 'disagree', 'statement', 'media', 'ruin', 'israels', 'reputation', 'rediculous', 'media', 'israeli', 'media', 'world', 'lived', 'europe', 'realize', 'incidences', 'described', 'letter', 'occured', 'media', 'whole', 'seem', 'ignore', 'subsidizing', 'israels', 'existance', 'europeans', 'least', 'degree', 'think', 'might', 'reason', 'report', 'clearly', 'atrocities', 'shame', 'austria', 'daily', 'reports', 'inhuman', 'acts', 'commited', 'israeli', 'soldiers', 'blessing', 'received', 'government', 'makes', 'holocaust', 'guilt', 'away', 'look', 'jews', 'treating', 'races', 'power', 'unfortunate'])
 list(['yeah', 'expect', 'people', 'read', 'actually', 'accept', 'hard', 'atheism', 'need', 'little', 'leap', 'faith', 'jimmy', 'logic', 'runs', 'steam', 'sorry', 'pity', 'sorry', 'feelings', 'denial', 'faith', 'need', 'well', 'pretend', 'happily', 'ever', 'anyway', 'maybe', 'start', 'newsgroup', 'atheist', 'hard', 'bummin', 'much', 'forget', 'flin

In [10]:
print(encoded[:2])

[[9, 59, 603, 207, 3278, 1495, 474, 702, 9470, 13686, 5533, 15227, 702, 442, 702, 70, 1148, 1095, 1036, 20294, 984, 705, 4294, 702, 217, 207, 1979, 15228, 13686, 4865, 4520, 87, 1530, 6, 52, 149, 581, 661, 4406, 4988, 4866, 1920, 755, 10668, 1102, 7837, 442, 957, 10669, 634, 51, 228, 2669, 4989, 178, 66, 222, 4521, 6066, 68, 4295], [1026, 532, 2, 60, 98, 582, 107, 800, 23, 79, 4522, 333, 7838, 864, 421, 3825, 458, 6488, 458, 2700, 4730, 333, 23, 9, 4731, 7262, 186, 310, 146, 170, 642, 1260, 107, 33568, 13, 985, 33569, 33570, 9471, 11491]]


In [11]:
vocab_size = len(word2idx) + 1
print('단어 집합의 크기: ', vocab_size)

단어 집합의 크기:  64277


네거티브 샘플링을 통한 데이터셋 구성하기

In [18]:
#네거티브 샘플링을 위한 케라스에서 제공하는 전처리 도구
from tensorflow.keras.preprocessing.sequence import skipgrams
#상위 10개의 뉴스그룹 샘플로 skipgram방식을 이용한 네거티브 샘플링 데이터 구성
skip_grams = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample
              in encoded[:10]]

[([[70, 61734], [474, 207], [603, 4348], [2669, 634], [661, 149], [4989, 957], [5533, 5861], [702, 4294], [52, 50568], [702, 1148], [702, 58798], [1530, 1920], [755, 21046], [10668, 1920], [207, 702], [661, 15228], [4521, 4295], [207, 8501], [4865, 20294], [10668, 34659], [474, 1495], [13686, 16260], [661, 56400], [3278, 62364], [755, 1530], [222, 35205], [4989, 43437], [15228, 24675], [634, 68], [4865, 50353], [1095, 58254], [13686, 59537], [178, 442], [51, 28741], [207, 24220], [4866, 957], [705, 19227], [4294, 16237], [442, 20763], [228, 4989], [702, 70], [13686, 1530], [442, 222], [15227, 42439], [1036, 16130], [1495, 442], [4520, 62043], [1495, 59], [1095, 207], [702, 4294], [702, 63789], [9, 38645], [4865, 4640], [661, 6], [222, 4989], [15228, 52583], [1095, 702], [1102, 63031], [9, 5533], [442, 2669], [603, 30060], [4406, 56297], [87, 4294], [51, 7837], [4406, 1083], [7837, 149], [5533, 207], [20294, 702], [634, 4989], [755, 10669], [15228, 1530], [1036, 5533], [1530, 30609], [7

In [20]:
#전체 뉴스 샘플 skip_grams 데이터 형태 확인
print(skip_grams)
#일부 뉴스 샘플 skip_grams 데이터 확인
print(skip_grams[11])

([[70, 61734],
  [474, 207],
  [603, 4348],
  [2669, 634],
  [661, 149],
  [4989, 957],
  [5533, 5861],
  [702, 4294],
  [52, 50568],
  [702, 1148],
  [702, 58798],
  [1530, 1920],
  [755, 21046],
  [10668, 1920],
  [207, 702],
  [661, 15228],
  [4521, 4295],
  [207, 8501],
  [4865, 20294],
  [10668, 34659],
  [474, 1495],
  [13686, 16260],
  [661, 56400],
  [3278, 62364],
  [755, 1530],
  [222, 35205],
  [4989, 43437],
  [15228, 24675],
  [634, 68],
  [4865, 50353],
  [1095, 58254],
  [13686, 59537],
  [178, 442],
  [51, 28741],
  [207, 24220],
  [4866, 957],
  [705, 19227],
  [4294, 16237],
  [442, 20763],
  [228, 4989],
  [702, 70],
  [13686, 1530],
  [442, 222],
  [15227, 42439],
  [1036, 16130],
  [1495, 442],
  [4520, 62043],
  [1495, 59],
  [1095, 207],
  [702, 4294],
  [702, 63789],
  [9, 38645],
  [4865, 4640],
  [661, 6],
  [222, 4989],
  [15228, 52583],
  [1095, 702],
  [1102, 63031],
  [9, 5533],
  [442, 2669],
  [603, 30060],
  [4406, 56297],
  [87, 4294],
  [51, 7837],
  

In [23]:
#첫번째 샘플인 skip_grams[0] 내 skipgrams로 형성된 데이터셋 확인
#윈도우 크기 내에서 중심 단어와 주변 단어의 관계이면 1의 레이블, 그렇지 않으면 0의 레이블을 가지도록 하여 데이터 셋 구성
pairs, labels = skip_grams[0][0], skip_grams[0][1]
print(labels)
for i in range(5):
    print("({:s} ({:d}), {:s}, ({:d})) -> {:d}".format(
        idx2word[pairs[i][0]], pairs[i][0],
        idx2word[pairs[i][1]], pairs[i][1],
        labels[i]))   

[0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 

In [22]:
print('전체 샘플 수: ', len(skip_grams))

전체 샘플 수:  10


In [24]:
# 첫번째 뉴스그룹 샘플에서 생긴 pairs와 labels의 개수
print(len(pairs))
print(len(labels))

2220
2220


In [25]:
#이 작업을 모든 뉴스그룹 샘플에 대해서 수행
skip_grams = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample in encoded]

Skip-Gram with Negative Sampling(SGNS) 구현하기

In [27]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input
from tensorflow.keras.layers import Dot
from tensorflow.keras.utils import plot_model
from IPython.display import SVG

In [28]:
#임베딩 벡터 차원은 100으로 정하고, 두개의 임베딩 층 추가
embedding_dim = 100

#중심 단어를 위한 임베딩 테이블
w_inputs = Input(shape=(1, ), dtype='int32')
word_embedding = Embedding(vocab_size, embedding_dim)(w_inputs)

#주변 단어를 위한 임베딩 테이블
c_inputs = Input(shape=(1, ), dtype='int32')
context_embedding = Embedding(vocab_size, embedding_dim)(c_inputs)

2023-03-05 22:50:21.826501: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [29]:
#각 단어는 임베딩 테이블을 거쳐서 내적을 수행하고,
#내적 결과는 1또는 0을 예측하기 위해 활성화 함수인 시그모이드 함수를 거쳐 최종 예측값을 획득
dot_product = Dot(axes=2)([word_embedding, context_embedding])
dot_product = Reshape((1,), input_shape=(1, 1))(dot_product)
output = Activation('sigmoid')(dot_product)

model = Model(inputs=[w_inputs, c_inputs], outputs=output)
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam')
plot_model(model, to_file='model3.png', show_shapes=True, show_layer_names=True, rankdir='TB')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 1, 100)       6427700     ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 1, 100)       6427700     ['input_2[0][0]']                
                                                                                              

In [30]:
#epoch = 5 
for epoch in range(1, 6):
    loss = 0
    for _, elem in enumerate(skip_grams):
        first_elem = np.array(list(zip(*elem[0]))[0], dtype='int32')
        second_elem = np.array(list(zip(*elem[0]))[1], dtype='int32')
        labels = np.array(elem[1], dtype='int32')
        X = [first_elem, second_elem]
        Y = labels
        loss += model.train_on_batch(X,Y)
    print('Epoch: ', epoch, 'Loss: ', loss)

Epoch:  1 Loss:  4631.042353525758
Epoch:  2 Loss:  3664.2201933301985


결과 확인하기

In [ ]:
import gensim

#학습된 임베딩 벡터들을 vecort.txt에 저장
f = open('vectors.txt', 'w')
f.write('{} {}\n'.format(vocab_size-1, embed_size))
vectors = model.get_weights()[0]
for word, i in tokenizer.word_index.items():
    f.write('{} {}\n'.format(word, ' '.join(map(str, list(vectors[i, :])))))
    f.close()

#모델 로드
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors.txt', binary=False)

In [ ]:
w2v.most_similar(positive=['soldiers'])